In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import math
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.svm import LinearSVC
import pickle
import time
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import Ridge

In [6]:
file_path = 'train.dat'
data = np.loadtxt(file_path)

In [7]:
# data_trn, data_tst = train_test_split( data, train_size = 30000 )

In [8]:
def my_map(X_raw):

    # Transform X_raw into d using element-wise computation
    d = np.flip(2 * X_raw - 1, axis=1)

    # Compute X using cumulative product along the flipped axis
    X = np.cumprod(d, axis=1)

    X_new = [X]
    for i in range(32):
        for j in range(i + 1, 32):
            prod_col = X[:, i] * X[:, j]
            X_new.append(prod_col[:, np.newaxis])

    X_new = np.concatenate(X_new, axis=1)
    return X_new


In [9]:
strt_time = time.time()
#X_trn, y_trn = my_map(data)
end_time = time.time()
print(end_time-strt_time)

ValueError: too many values to unpack (expected 2)

In [ ]:
for i in range(1):
    print(f"Data Point {i} - X: {X_trn[i, :]}, y: {y_trn[i]}")

In [ ]:
def reduce_dimensionality_with_lasso(X_trn, y_trn, X_test, y_test, alpha, threshold=1e-4):
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_trn, y_trn)
    coefficients = lasso_model.coef_
    selected_features = np.where(np.abs(coefficients) > threshold)[0]
    X_trn_reduced = X_trn[:, selected_features]
    X_test_reduced = X_test[:, selected_features]
    return X_trn_reduced, X_test_reduced

In [ ]:
def reduce_dimensionality_with_ridge(X_trn, y_trn, X_test, y_test, alpha, threshold=1e-4):
    ridge_model = Ridge(alpha=alpha)
    ridge_model.fit(X_trn, y_trn)
    coefficients = ridge_model.coef_
    selected_features = np.where(np.abs(coefficients) > threshold)[0]
    X_trn_reduced = X_trn[:, selected_features]
    X_test_reduced = X_test[:, selected_features]
    return X_trn_reduced, X_test_reduced


In [ ]:
file_path_2 = '/content/test.dat'
test_data = np.loadtxt(file_path_2)

In [ ]:
X_test, y_test = my_map(test_data)

In [ ]:
X_trn_new,X_test_new = reduce_dimensionality_with_lasso(X_trn, y_trn, X_test, y_test, alpha=0.005, threshold=1e-4)

In [ ]:
# X_trn_new,X_test_new = reduce_dimensionality_with_ridge(X_trn, y_trn, X_test, y_test, alpha=0.1, threshold=1e-4)

In [ ]:
print( X_test.shape[1])
print(X_trn_new.shape[1])
print(X_test_new.shape[1])

In [ ]:
def my_fit(X, y):
    X_raw=my_map(X)
    classifier = LogisticRegression(random_state=0)
    classifier.fit(X_raw, y)
    return classifier

In [ ]:
strt_time = time.time()
cls_1 = my_fit(X_trn,y_trn)
end_time = time.time()
print(end_time-strt_time)

In [ ]:
y_pred = cls_1.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

In [ ]:

cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
cls_2 = my_fit(X_trn_new,y_trn)

In [ ]:
y_pred_new = cls_2.predict(X_test_new)
print(np.concatenate((y_pred_new.reshape(len(y_pred_new),1),y_test.reshape(len(y_test),1)),1))

In [ ]:
cm_2 = confusion_matrix(y_test, y_pred_new)
print(cm_2)
accuracy_score(y_test, y_pred_new)